In [ ]:
from jinja2 import Environment, FileSystemLoader
import numpy as np

In [ ]:
opt_f='bromide.log'
not_opt_f='coumarin.log'
with open(not_opt_f, 'r') as f:
    lines = f.readlines()

In [ ]:
# Check if optimization was succesfull 
opt = False
for line in lines:
    if  "**** THE GEOMETRY SEARCH IS NOT CONVERGED! ****" in line:
        opt = True

In [ ]:
# If optimization is successfull 
if not opt:
    data = [] 
    for num, line in enumerate(lines):
        if "TOTAL NUMBER OF ATOMS" in line:
            n_atoms = int(line.split()[5])
        if "EQUILIBRIUM" in line:
            start_coord_index = num + 4
            end_coord_index = start_coord_index + n_atoms
            for i in range(start_coord_index, end_coord_index):
                data.append(lines[i].split())
                
    # store opt coordinates into a list
    atomic_number = []
    element_symbol = []
    x_coord = []
    y_coord = []
    z_coord = []
    for line in data:
        element_symbol.append(line[0])
        atomic_number.append(line[1])
        x_coord.append(line[2])
        y_coord.append(line[3])
        z_coord.append(line[4])
    opt_coord = (element_symbol, x_coord, y_coord, z_coord)
    result = np.transpose(np.array(opt_coord))

In [ ]:
# Option 1
file_loader=FileSystemLoader("../templates")
environment = Environment(loader=file_loader)
if not opt:

    template = environment.get_template("structure_input_template.xyz")

    MOLNAME="bromide"
    content = template.render(N_ATOMS=n_atoms, MOLNAME=MOLNAME, COORDINATES=result)
    print(content)
    
    # write into a file
    # filename = MOLNAME+'.xyz'
    # with open(filename, mode="w", encoding="utf-8") as message:
    #    message.write(content)

In [ ]:
# Option 2    

# opt_xyz = open(MOLNAME+'.xyz','w')
# opt_xyz.write(str(n_atoms) +'\n')
# opt_xyz.write(MOLNAME +'\n')
if not opt:
        print(MOLNAME +'\n')
        print(str(n_atoms) +'\n')
        for i in range(n_atoms):
                line="{0:<4s}".format(element_symbol[i]) + ' ' 
                line+="{0:>16.10f}".format(float(x_coord[i])) + ' '
                line+="{0:>16.10f}".format(float(y_coord[i])) + ' '
                line+="{0:>16.10f}".format(float(z_coord[i])) + ' ' + '\n'
                print(line)
#         opt_xyz.write(line)
# opt_xyz.close()

In [ ]:
# If optimization is NOT successfull 

if  opt:
	template = environment.get_template("gamess_input_template.inp")
	data = []
	for num, line in enumerate(lines):
		if "TOTAL NUMBER OF ATOMS" in line:
			n_atoms = int(line.split()[5])
		if "ALWAYS THE LAST POINT COMPUTED!" in line:
			start_coord_index = num + 4
			end_coord_index = start_coord_index + n_atoms
			for i in range(start_coord_index, end_coord_index):
				data.append(lines[i])
				
	with open("gamess_parameters.in", 'r') as f:
		lines = f.readlines()
	for line in lines:
		if "SCFTYP" in line:
			SCFTYP = line.split()[2]
		if "MULT" in line:
			MULT = line.split()[2]
		if "OPTTOL" in line:
			OPTTOL = line.split()[2]
		if "NSTEP" in line:
			NSTEP = line.split()[2]
		if "GBASIS" in line:
			GBASIS = line.split()[2]
		if "NGAUSS" in line:
			NGAUSS = line.split()[2]
		if "MOLNAME" in line:
			MOLNAME = line.split()[2]


	content = template.render(SCFTYP=SCFTYP, MULT=MULT, OPTTOL=OPTTOL,
			NSTEP=NSTEP, GBASIS=GBASIS, NGAUSS=NGAUSS, MOLNAME=MOLNAME, COORDINATES=data)
	print(content)
	
	# write into a file
	# filename = MOLNAME+'.inp'
	# with open(filename, mode="w", encoding="utf-8") as message:
	# 	message.write(content)

In [ ]:
with open("cluster_parameters.in", 'r') as f:
    lines = f.readlines()
for line in lines:
    if "WALLTIME" in line:
        WALLTIME = line.split()[2]
    if "BIN" in line:
        BIN = line.split()[2]
    if "USER_EMAIL" in line:
        USER_EMAIL = line.split()[2]

template = environment.get_template("array_sub_template.sh")

content = template.render(WALLTIME=WALLTIME, BIN=BIN, USER_EMAIL=USER_EMAIL)
print(content)